## Query and edit Database

In [ ]:
import requests
import json
data = {
    "items": [
        "https://www.linkedin.com/in/jude-samuel-3984384/", 
        "https://www.linkedin.com/in/zain-n-112a361a8/"
        ]
}
endpoint = "https://hook.us1.make.com/4e9915dpiack58vd7mqc5rfv2y8ahjsw"
attempt = requests.post(url = endpoint, headers={"Content-Type": "application/json"}, data=json.dumps(data))

print(attempt.text)

Accepted


### Class

In [4]:
import os
from dotenv import load_dotenv
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

class MongoDB:
    """
    Connection to MongoDB Database
    """
    def __init__(self): # initialise connection to DB
        load_dotenv()
        uri = os.getenv("MONGODBURI")
        self.uri = uri
        # Create a new client and connect to the server
        self.client = MongoClient(uri, server_api=ServerApi('1'))

        # Send a ping to confirm a successful connection
        try:
            self.client.admin.command('ping')
            print("Pinged your deployment. You successfully connected to MongoDB!")

        except Exception as e:
            print(e)
        
        
        self.db = self.client.get_database("Contacts")
        self.collection = self.db.get_collection("URLs")
    

    def close(self):
        if self.client:
            self.client.close()
            print("MongoDB Connection closed")

    def search(self, key, value): # Search for key-value pair

        client = MongoClient(self.uri)
        try:
            query = { f"{key}": f"{value}" }
            movie = self.collection.find_one(query)
            return movie

        except Exception as e:
            raise Exception("Unable to find the document due to the following error: ", e)

    def add(self, record: dict): # Add dictionary to database
        try:
            self.db.URLs.insert_one(record)
            return True
        except Exception as e:
            raise Exception("Unable to add document due to the following error: ", e)

    def edit(self, sort: dict, replacement: dict): # Replace existing record with new one
        """
        filter: {"field": "value"}
        replacement: {"name": name, "link": link}
        """
        try:
            self.db.URLs.replace_one(sort, replacement)
            return True
        except Exception as e:
            raise Exception("Unable to edit document due to the following error: ", e)
    
    def filter(self, field: str): # Filter database by existence of field
        try:
            results = self.db.URLs.find({
                f"{field}": {"$exists": True}
                })
            return results
        except Exception as e:
            raise Exception("Unable to filter database due to the following error: ", e)

In [5]:
db = MongoDB()
filtered = db.filter("experience")

for point in filtered:
    print(point)


db.close()

Pinged your deployment. You successfully connected to MongoDB!
MongoDB Connection closed


# Playground

In [6]:
client = MongoDB()
record = client.search("name", "Zain Nomani")
print(record)

Pinged your deployment. You successfully connected to MongoDB!
{'_id': ObjectId('6897907bf8d4312bfa216ccf'), 'name': 'Zain Nomani', 'link': 'https://www.linkedin.com/in/zain-n-112a361a8/'}


In [7]:
client.add({
    "name": "Jeremy Clarkson",
    "link": "POWERRRRR"
    })


True

In [9]:
client.edit({"name": "Jeremy Clarkson"}, replacement = {"name": "Jeremy Clarkson", "link": "https://www.linkedin.com/in/orangutan/"})

True

In [7]:
load_dotenv()
uri = os.getenv("MONGODBURI")
uri = uri
# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
    client = MongoClient(uri)
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


In [12]:
db = client.get_database("Contacts")
results = db.URLs.find({"name": {"$exists": True}})

for result in results:
    print(result)

{'_id': ObjectId('6897907bf8d4312bfa216ccf'), 'name': 'Zain Nomani', 'link': 'https://www.linkedin.com/in/zain-n-112a361a8/'}
{'_id': ObjectId('689796cd8df42e987bec26f4'), 'name': 'Tim Campbell', 'link': 'https://www.linkedin.com/in/timcampbellmbe/'}
{'_id': ObjectId('6897973c8df42e987bec26f6'), 'name': 'Richard Branson', 'link': 'https://www.linkedin.com/in/rbranson/'}
{'_id': ObjectId('68979cdfea840bef038dc64c'), 'name': 'Jeremy Clarkson', 'link': 'https://www.linkedin.com/in/orangutan/'}
